In [2]:
import pandas as pd
import regex as re
from bs4 import BeautifulSoup
from bertopic.representation import KeyBERTInspired, PartOfSpeech, MaximalMarginalRelevance
from bertopic import BERTopic



import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

nltk.download('wordnet')
nltk.download("stopwords")
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('maxent_ne_chunker_tab')
nltk.download('words')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/cs/grad/opumni/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cs/grad/opumni/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/cs/grad/opumni/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/cs/grad/opumni/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker_tab to
[nltk_data]     /home/cs/grad/opumni/nltk_data...
[nltk_data]   Package maxent_ne_chunker_tab is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /home/cs/grad/opumni/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [ ]:
def decontracted(phrase):
    if not isinstance(text, str):
        return ""
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def replace_file_paths(text):
    if not isinstance(text, str):
        return ""
    # Regex pattern for detecting file paths (both Linux and Windows)
    # Matches Windows paths (C:\path\to\file) and Linux paths (/path/to/file)
    pattern = r'([A-Za-z]:\\(?:[\w\-\s]+\\)*[\w\-\s]+\.[a-zA-Z0-9]+)|(/\S+)'

    # Replace file paths with 'PATH'
    cleaned_text = re.sub(r'https?://[\n\S]+\b', 'URL', text)
    cleaned_text = re.sub(pattern, 'PATH', cleaned_text)
    return cleaned_text

def replace_code_identifiers(text):
    if not isinstance(text, str):
        return ""
    # Regex pattern to match function calls, compound identifiers, and camelCase names
    pattern = r'\b[A-Za-z_][A-Za-z0-9_.\-]*\(\)|\b[A-Z_]+:[A-Za-z0-9_]+|\b[a-zA-Z_][a-zA-Z0-9]*[A-Z][a-zA-Z0-9]*'

    
    cleaned_text = re.sub("\w+_\w+", 'CODE', text)
    
    # Replace matches with 'CODE'
    cleaned_text = re.sub(pattern, 'CODE', cleaned_text)

    return cleaned_text

def replace_numbers(text):
    if not isinstance(text, str):
        return ""
    cleaned_text = re.sub(r'\d+', ' NUMBER ', text)

    return cleaned_text

def replace_quoted_text(text):
    if not isinstance(text, str):
        return ""
    text = text.replace('"','\'')
    text = text.replace('`','\'')

    # Regex pattern to match text inside single, double, and backticks but not apostrophes inside words
    pattern = r'([\'"`])[^\'"`]*?\1'

    # Replace quoted text with 'TEXT'
    cleaned_text = re.sub(pattern, ' QUOTE ', text)

    return cleaned_text

def remove_commands(text):
    if not isinstance(text, str):
        return ""
    # Define lists of common Linux and Windows commands
    linux_commands = ['ls', 'cat', 'cd', 'grep', 'mkdir', 'rm', 'touch', 'pwd', 'chmod', 'cp', 'mv']
    windows_commands = ['dir', 'cls', 'copy', 'del', 'echo', 'mkdir', 'rmdir', 'type', 'move', 'ren']

    # Combine the commands into a single list
    all_commands = linux_commands + windows_commands

    # Escape commands to make sure they are valid regex patterns
    escaped_commands = [re.escape(cmd) for cmd in all_commands]

    # Create a regex pattern to match any of the commands
    pattern = r'\b(?:' + '|'.join(escaped_commands) + r')\b'
    # Replace commands with an empty string
    cleaned_text = re.sub(pattern, ' COMMAND ', text)

    return cleaned_text

In [3]:
ISSUE_DATA_PATH_NEW = "./Data/AllIssues-New.csv"
ISSUE_DATA_PATH_OLD = "./Data/AllIssues-Old-Actives.csv"
df1 = pd.read_csv(ISSUE_DATA_PATH_NEW, low_memory=False)
df1['Body'] = df1['Body'].fillna('')

df2 = pd.read_csv(ISSUE_DATA_PATH_OLD, low_memory=False)
df2['Body'] = df2['Body'].fillna('')

df = pd.concat([df1,df2], ignore_index=True).reset_index(drop=True)

df['combinedText'] = df['Title'] + ' ' + df['Body']

#df["combinedText"] = df["combinedText"].apply(lambda x: decontracted(x))
#df["combinedText"] = df["combinedText"].apply(lambda x: remove_commands(x))
#df["combinedText"] = df["combinedText"].apply(lambda x: replace_file_paths(x))
#df["combinedText"] = df["combinedText"].apply(lambda x: replace_code_identifiers(x))
#df["combinedText"] = df["combinedText"].apply(lambda x: replace_quoted_text(x))
#df["combinedText"] = df["combinedText"].apply(lambda x: replace_numbers(x))

# punc_tokenizer = RegexpTokenizer(r'\w+')
# df["combinedText"] = df["combinedText"].apply(lambda x: " ".join(punc_tokenizer.tokenize(x)))

# stop_words = set(stopwords.words('english'))
# df["combinedText"] = df["combinedText"].apply(lambda x:" ".join([w for w in word_tokenize(x) if not w.lower() in stop_words]))

# lemmatizer = WordNetLemmatizer()
# df["combinedText"] = df["combinedText"].apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in  word_tokenize(x)]))

df = df[df['combinedText'].notna()]
data = df['combinedText'].tolist()

In [ ]:
#representation_model = [KeyBERTInspired(), MaximalMarginalRelevance(diversity=0.3)]

#umap_model = UMAP(n_neighbors=15, n_components=100, metric='cosine', low_memory=False)
#hdbscan_model = HDBSCAN(min_cluster_size=20, metric='euclidean', prediction_data=True)
#umap_model=umap_model,hdbscan_model=hdbscan_model, 
topic_model = BERTopic(calculate_probabilities=False, nr_topics='auto', language="english", )
topics, probs = topic_model.fit_transform(data)

In [ ]:
hierarchical_topics = topic_model.hierarchical_topics(data)

100%|██████████| 6/6 [00:00<00:00, 13.95it/s]


In [ ]:
topic_model.get_topic_info()

NameError: name 'topic_model' is not defined

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [ ]:
topic_model.visualize_heatmap()